Evalvuate the eror 
The objective of the eval pred is to first figure out some info about the evaluation metric. The evaluation metric Consists of two parts. Ftist is a rank based one. We rank the customers that have defulted based on the predicted probability. choose the highest .04(#1 + #0*20). 

This is equvalent to the recall level of 4% the doc states. That is, of all the customers that are going to deafult, we aim to recall at least 96% of them!! 


# Q:
 - What does a recall of 4% mean? How do we train for a recall of a 4%? 

In [4]:
import pandas as pd
from pd.params import *

In [8]:
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

from pd.nn.model import Conv
from pd.data.loader import CustomerData, DTwithLabelRatio
from pd.nn.train_utils import train_torch_model
from pd.metric import amex_metric
from pd.params import *
from pd.pred import pred_test_npy as predict

In [5]:
def amex_metric(y_true, y_pred, return_components=False) -> float:
    """Amex metric for ndarrays"""
    def top_four_percent_captured(df) -> float:
        """Corresponds to the recall for a threshold of 4 %"""
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
        
    def weighted_gini(df) -> float:
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(df) -> float:
        """Corresponds to 2 * AUC - 1"""
        df2 = pd.DataFrame({'target': df.target, 'prediction': df.target})
        df2.sort_values('prediction', ascending=False, inplace=True)
        return weighted_gini(df) / weighted_gini(df2)

    df = pd.DataFrame({'target': y_true.ravel(), 'prediction': y_pred.ravel()})
    df.sort_values('prediction', ascending=False, inplace=True)
    g = normalized_weighted_gini(df)
    d = top_four_percent_captured(df)

    if return_components: return g, d, 0.5 * (g + d)
    return 0.5 * (g + d)

In [6]:
def top_four_percent_captured(df) -> float:
        """Corresponds to the recall for a threshold of 4 %"""
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
    

In [3]:
import numpy as np
import torch 
from pd.nn.model import Conv

In [2]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
from bes.nn.es_module import ESModule


In [7]:
train_data = np.load(OUTDIR+"c13_data.npy")
train_labels = np.load(OUTDIR+"c13_labels.npy")

In [10]:
X_train, X_test, y_train, y_test = train_test_split(train_data, train_labels, test_size=1/9, random_state=0, shuffle=True)
validation_data = (X_test, y_test)

train_dataset = CustomerData(X_train, train_labels=y_train)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE)


In [14]:
model_name = "conv_all_13"
model = Conv(input_dim=X_train.shape[-1], conv_channels=25)
model = train_torch_model(model, train_loader, num_epochs=50, validation_data=validation_data, 
                        output_model_name=model_name)
 

0, BCE loss: 0.746, amex train: 0.033, val 0.000
0, BCE loss: 0.880, amex train: 0.146, val 0.000
0, BCE loss: 0.725, amex train: 0.220, val 0.000
0, BCE loss: 0.568, amex train: 0.288, val 0.000
0, BCE loss: 0.526, amex train: 0.314, val 0.000
0, BCE loss: 0.582, amex train: 0.345, val 0.000
0, BCE loss: 0.607, amex train: 0.355, val 0.000
0, BCE loss: 0.571, amex train: 0.358, val 0.000
0, BCE loss: 0.529, amex train: 0.384, val 0.000
0, BCE loss: 0.511, amex train: 0.386, val 0.000
0, BCE loss: 0.519, amex train: 0.391, val 0.000
0, BCE loss: 0.531, amex train: 0.378, val 0.000
0, BCE loss: 0.546, amex train: 0.387, val 0.000
0, BCE loss: 0.537, amex train: 0.388, val 0.000
0, BCE loss: 0.542, amex train: 0.388, val 0.000
0, BCE loss: 0.529, amex train: 0.396, val 0.000
0, BCE loss: 0.520, amex train: 0.399, val 0.000
0, BCE loss: 0.494, amex train: 0.387, val 0.000
1, BCE loss: 0.507, amex train: 0.408, val 0.000
1, BCE loss: 0.503, amex train: 0.387, val 0.000
1, BCE loss: 0.511, 

: 

: 

(343141, 13, 188)